<a href="https://colab.research.google.com/github/z9manuel/hacking-civico/blob/master/tareas/03_exploracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nota:** No me funciono la descarga, los baje directo de la página y luego subi los datos de forma manual.

In [ ]:
# Para descargar el conjunto de datos
!wget http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

--2020-09-09 21:31:26--  http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 187.210.186.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|187.210.186.146|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30020764 (29M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip’

datos_abiertos_covi 100%[===================>]  28.63M  9.66MB/s    in 3.0s    

2020-09-09 21:31:29 (9.66 MB/s) - ‘datos_abiertos_covid19.zip’ saved [30020764/30020764]



In [6]:
import statistics

import pandas as pd
import numpy as np

In [11]:
# Especificamos el nombre del archivo por abrir
archivo = 'COVID19MEXICO.csv'

# Utilizamos la función de Pandas para cargar un archivo CSV
data = pd.read_csv(archivo, encoding='latin-1')

In [12]:
data.head()


,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-09-08,1413b4,2,4,25,1,25,25,6,1,2020-04-29,2020-04-24,9999-99-99,97,2,38,1,2,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
1,2020-09-08,18f860,2,3,2,1,2,2,2,2,2020-05-27,2020-05-27,9999-99-99,2,1,59,1,2,2,1,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,2
2,2020-09-08,1057de,2,3,15,2,15,15,33,1,2020-06-19,2020-06-16,9999-99-99,97,2,33,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
3,2020-09-08,18ae31,2,4,15,1,15,15,58,1,2020-04-20,2020-04-15,9999-99-99,97,2,34,1,2,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,97
4,2020-09-08,02d970,2,4,13,2,13,13,76,2,2020-04-27,2020-04-23,2020-04-27,2,2,61,1,97,2,2,1,2,2,1,2,2,2,2,2,99,1,99,MÃ©xico,99,2


In [13]:
# Cargar el archivo con los descriptores.
nombre_diccionario = 'Descriptores_0419.xlsx'
diccionario = pd.read_excel(nombre_diccionario)

In [14]:
# Mostramos diccionario
diccionario

,Nº,NOMBRE DE VARIABLE,DESCRIPCIÓN DE VARIABLE,FORMATO O FUENTE
0,1,FECHA_ACTUALIZACION,"La base de datos se alimenta diariamente, esta...",AAAA-MM-DD
1,2,ID_REGISTRO,Número identificador del caso,TEXTO
2,3,ORIGEN,La vigilancia centinela se realiza a través de...,CATÁLOGO: ORIGEN ...
3,4,SECTOR,Identifica el tipo de institución del Sistema ...,CATÁLOGO: SECTOR ...
4,5,ENTIDAD_UM,Identifica la entidad donde se ubica la unidad...,CATALÓGO: ENTIDADES
5,6,SEXO,Identifica al sexo del paciente.,CATÁLOGO: SEXO
6,7,ENTIDAD_NAC,Identifica la entidad de nacimiento del paciente.,CATALÓGO: ENTIDADES
7,8,ENTIDAD_RES,Identifica la entidad de residencia del paciente.,CATALÓGO: ENTIDADES
8,9,MUNICIPIO_RES,Identifica el municipio de residencia del paci...,CATALÓGO: MUNICIPIOS
9,10,TIPO_PACIENTE,Identifica el tipo de atención que recibió el ...,CATÁLOGO: TIPO_PACIENTE


In [16]:
# Cargamos archivo de catalogos
nombre_cat = 'Catalogos_0412.xlsx'
sheets = ['ORIGEN', 'SECTOR', 'SEXO', 'TIPO_PACIENTE', 'SI_NO', 'NACIONALIDAD', 'RESULTADO', 'de ENTIDADES', 'MUNICIPIOS']
catalogues = {}
for sheet in sheets:
  catalogue = pd.read_excel(nombre_cat, sheet_name='Catálogo ' + sheet)
  catalogue.columns = catalogue.columns.str.replace(' ', '_')
  catalogue.columns = catalogue.columns.str.replace('.', '')
  catalogue.columns = catalogue.columns.str.lower()
  new_catalogue = {sheet: catalogue}
  catalogues.update(new_catalogue)

In [17]:
# Mostramos cátalogos
catalogues

{'MUNICIPIOS':       clave_municipio        municipio  clave_entidad
 0                   1   AGUASCALIENTES              1
 1                   2         ASIENTOS              1
 2                   3         CALVILLO              1
 3                   4            COSÍO              1
 4                   5      JESÚS MARÍA              1
 ...               ...              ...            ...
 2496              999  NO ESPECIFICADO             32
 2497              999  NO ESPECIFICADO             36
 2498              997        NO APLICA             97
 2499              998        SE IGNORA             98
 2500              999  NO ESPECIFICADO             99
 
 [2501 rows x 3 columns], 'NACIONALIDAD':    clave      descripción
 0      1         MEXICANA
 1      2       EXTRANJERA
 2     99  NO ESPECIFICADO, 'ORIGEN':    clave      descripción
 0      1            USMER
 1      2   FUERA DE USMER
 2     99  NO ESPECIFICADO, 'RESULTADO':   unnamed:_0              unnamed:_1
 0    

In [18]:
# Ejemplo de un catalgo
catalogues['SEXO']


,clave,descripción
0,1,MUJER
1,2,HOMBRE
2,99,NO ESPECIFICADO


## **Perfilamiento**
**Descripción General**


In [19]:
# dimensión de los datos
print('Número de filas: {}'.format(data.shape[0]))
print('Número de columnas: {}'.format(data.shape[1]))

Número de filas: 1450397
Número de columnas: 35


In [20]:
# columnas (variables)
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [22]:
# limpieza del nombre de las columnas, remover espacios, carácteres especiales y pasar a minúsculas
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('.', '')
data.columns = data.columns.str.lower()

In [23]:
# calcular número de valores nulos por columna
for col in data.columns:
  print('Valores nulos en "{}": {}'.format(col,data[col].isna().sum()))

Valores nulos en "fecha_actualizacion": 0
Valores nulos en "id_registro": 0
Valores nulos en "origen": 0
Valores nulos en "sector": 0
Valores nulos en "entidad_um": 0
Valores nulos en "sexo": 0
Valores nulos en "entidad_nac": 0
Valores nulos en "entidad_res": 0
Valores nulos en "municipio_res": 0
Valores nulos en "tipo_paciente": 0
Valores nulos en "fecha_ingreso": 0
Valores nulos en "fecha_sintomas": 0
Valores nulos en "fecha_def": 0
Valores nulos en "intubado": 0
Valores nulos en "neumonia": 0
Valores nulos en "edad": 0
Valores nulos en "nacionalidad": 0
Valores nulos en "embarazo": 0
Valores nulos en "habla_lengua_indig": 0
Valores nulos en "diabetes": 0
Valores nulos en "epoc": 0
Valores nulos en "asma": 0
Valores nulos en "inmusupr": 0
Valores nulos en "hipertension": 0
Valores nulos en "otra_com": 0
Valores nulos en "cardiovascular": 0
Valores nulos en "obesidad": 0
Valores nulos en "renal_cronica": 0
Valores nulos en "tabaquismo": 0
Valores nulos en "otro_caso": 0
Valores nulos 

In [24]:
# tipos de datos
data.dtypes

fecha_actualizacion    object
id_registro            object
origen                  int64
sector                  int64
entidad_um              int64
sexo                    int64
entidad_nac             int64
entidad_res             int64
municipio_res           int64
tipo_paciente           int64
fecha_ingreso          object
fecha_sintomas         object
fecha_def              object
intubado                int64
neumonia                int64
edad                    int64
nacionalidad            int64
embarazo                int64
habla_lengua_indig      int64
diabetes                int64
epoc                    int64
asma                    int64
inmusupr                int64
hipertension            int64
otra_com                int64
cardiovascular          int64
obesidad                int64
renal_cronica           int64
tabaquismo              int64
otro_caso               int64
resultado               int64
migrante                int64
pais_nacionalidad      object
pais_orige

**Limpieza**

In [25]:
# convertimos a fecha algunas columnas
def cast_datetime_cols(df):
    timestamp_cols = [col for col in df.columns if "fecha" in col]
    df[timestamp_cols] = df[timestamp_cols].apply(lambda date_col: pd.to_datetime(date_col, errors="coerce"), axis=0)
    return df

data = cast_datetime_cols(data)

In [26]:

data.head()

,fecha_actualizacion,id_registro,origen,sector,entidad_um,sexo,entidad_nac,entidad_res,municipio_res,tipo_paciente,fecha_ingreso,fecha_sintomas,fecha_def,intubado,neumonia,edad,nacionalidad,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,resultado,migrante,pais_nacionalidad,pais_origen,uci
0,2020-09-08,1413b4,2,4,25,1,25,25,6,1,2020-04-29,2020-04-24,NaT,97,2,38,1,2,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
1,2020-09-08,18f860,2,3,2,1,2,2,2,2,2020-05-27,2020-05-27,NaT,2,1,59,1,2,2,1,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,2
2,2020-09-08,1057de,2,3,15,2,15,15,33,1,2020-06-19,2020-06-16,NaT,97,2,33,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
3,2020-09-08,18ae31,2,4,15,1,15,15,58,1,2020-04-20,2020-04-15,NaT,97,2,34,1,2,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,97
4,2020-09-08,02d970,2,4,13,2,13,13,76,2,2020-04-27,2020-04-23,2020-04-27,2,2,61,1,97,2,2,1,2,2,1,2,2,2,2,2,99,1,99,MÃ©xico,99,2


In [28]:
# unimos los datos con los catalogos para tener mejor entendimiento de ellos
data = (
    # unimos con el catálogo MUNICIPIOS y ENTIDADES
    data.merge(catalogues['de ENTIDADES'], how='left', left_on='entidad_res', right_on='clave_entidad').
    drop(columns=['entidad_res', 'abreviatura']).
    rename(columns={'entidad_federativa':'entidad_res'}).
    merge(catalogues['MUNICIPIOS'], how='left', left_on=['clave_entidad','municipio_res'], right_on=['clave_entidad','clave_municipio']).
    drop(columns=['municipio_res', 'clave_entidad', 'clave_municipio']).
    rename(columns={'municipio':'municipio_res'})
)

ValueError: ignored

In [29]:

# unimos con el catálogo de ENTIDADES
entidades_cols = [col for col in data.columns if "entidad" in col and col != 'entidad_res']
for col in entidades_cols:
    data = (
        data.merge(catalogues['de ENTIDADES'], how='left', left_on=col, right_on='clave_entidad').
        drop(columns=[col, 'abreviatura', 'clave_entidad']).
        rename(columns={'entidad_federativa':col})
    )

In [30]:
# unimos con el catálogo SI_NO
sino_cols = ['intubado', 'neumonia', 'embarazo', 'habla_lengua_indig', 'diabetes', 'epoc', 'asma', 'inmusupr',
             'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 
             'otro_caso', 'migrante', 'uci']
for col in sino_cols:
    data = (
        data.merge(catalogues['SI_NO'], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [31]:
# Corregimos nombre de columnas
catalogues['RESULTADO'].columns

Index(['unnamed:_0', 'unnamed:_1'], dtype='object')

In [32]:
catalogues['RESULTADO'].columns = ['clave', 'descripción']
catalogues['RESULTADO'].columns

Index(['clave', 'descripción'], dtype='object')

In [33]:
Removemos la primer fila que no es útil
catalogues['RESULTADO'] = catalogues['RESULTADO'].drop(index=0)

In [34]:
# Convertimos el tipo de dato
catalogues['RESULTADO']['clave'] = catalogues['RESULTADO']['clave'].astype(int)

In [35]:
# Fin dela limpieza
# unimos otras columnnas que comparten lógica
other_cols = ['origen', 'sector', 'sexo', 'tipo_paciente', 'nacionalidad', 'resultado']
for col, cat in zip(other_cols, [x.upper() for x in other_cols]):
    data = (
        data.merge(catalogues[cat], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )


In [36]:
# creamos algunas variables que podrían ser útiles después
data['defuncion'] = ~data.fecha_def.isna()
data['mes_def'] = data.fecha_def.dt.month
data['año_def'] = data.fecha_def.dt.year
data['mes_ingreso'] = data.fecha_ingreso.dt.month
data['año_ingreso'] = data.fecha_ingreso.dt.year

In [37]:
data.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-08,1413b4,2020-04-29,2020-04-24,NaT,38,MÃ©xico,99,SINALOA,CULIACÁN,SINALOA,SINALOA,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020
1,2020-09-08,18f860,2020-05-27,2020-05-27,NaT,59,MÃ©xico,99,BAJA CALIFORNIA,MEXICALI,BAJA CALIFORNIA,BAJA CALIFORNIA,NO,SI,NO,NO,SI,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO,FUERA DE USMER,ESTATAL,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020
2,2020-09-08,1057de,2020-06-19,2020-06-16,NaT,33,MÃ©xico,99,MÉXICO,ECATEPEC DE MORELOS,MÉXICO,MÉXICO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,6,2020
3,2020-09-08,18ae31,2020-04-20,2020-04-15,NaT,34,MÃ©xico,99,MÉXICO,NEZAHUALCÓYOTL,MÉXICO,MÉXICO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020
4,2020-09-08,02d970,2020-04-27,2020-04-23,2020-04-27,61,MÃ©xico,99,HIDALGO,TULA DE ALLENDE,HIDALGO,HIDALGO,NO,NO,NO APLICA,NO,NO,SI,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020


##**¿Tienen los pacientes con hipertensión un riesgo más alto de defunción?**

In [41]:
# hipertension
data.groupby('hipertension').defuncion.value_counts(normalize=True)


hipertension  defuncion
NO            False        0.960276
              True         0.039724
SE IGNORA     False        0.874874
              True         0.125126
SI            False        0.833960
              True         0.166040
Name: defuncion, dtype: float64

**Respuesta:** De los que no tenian hipertensión fallecio el 3.9724% mientras que de los que si tenian fallecio el 16.6040%

##**¿Cuántos casos confirmados se tienen por Estado?**

In [49]:
data.entidad_res.value_counts()

CIUDAD DE MÉXICO                   297270
MÉXICO                             178629
GUANAJUATO                          82439
NUEVO LEÓN                          74491
PUEBLA                              61427
TAMAULIPAS                          59580
TABASCO                             58103
JALISCO                             56616
COAHUILA DE ZARAGOZA                56397
VERACRUZ DE IGNACIO DE LA LLAVE     47492
SAN LUIS POTOSÍ                     43722
MICHOACÁN DE OCAMPO                 42085
SONORA                              41707
SINALOA                             30521
YUCATÁN                             30267
BAJA CALIFORNIA                     29084
GUERRERO                            27829
OAXACA                              21421
DURANGO                             21290
CHIHUAHUA                           20209
HIDALGO                             19265
BAJA CALIFORNIA SUR                 19054
AGUASCALIENTES                      18775
TLAXCALA                          

##**¿Cuántas defunciones se tienen por Estado?**

In [107]:


lista = data.groupby(['entidad_res'])['defuncion'].value_counts()
lista.head(60)

entidad_res                      defuncion
AGUASCALIENTES                   False         18226
                                 True            549
BAJA CALIFORNIA                  False         25065
                                 True           4019
BAJA CALIFORNIA SUR              False         18545
                                 True            509
CAMPECHE                         False         10667
                                 True            949
CHIAPAS                          False          9645
                                 True           1219
CHIHUAHUA                        False         18218
                                 True           1991
CIUDAD DE MÉXICO                 False        285463
                                 True          11807
COAHUILA DE ZARAGOZA             False         54345
                                 True           2052
COLIMA                           False          6465
                                 True            569
DUR

In [84]:
lista.tail(10)

entidad_res                      defuncion
TAMAULIPAS                       False        56801
                                 True          2779
TLAXCALA                         False        17208
                                 True          1309
VERACRUZ DE IGNACIO DE LA LLAVE  False        42765
                                 True          4727
YUCATÁN                          False        28499
                                 True          1768
ZACATECAS                        False        12400
                                 True           726
Name: id_registro, dtype: int64

## **¿Cuántos fallecimientos han ocurrido en el Estado con mayor número de casos confirmados?**

In [140]:
data[data.defuncion & (data.entidad_res == "CIUDAD DE MÉXICO")].defuncion.value_counts()

True    11807
Name: defuncion, dtype: int64

##**¿Cuántos fallecimientos han ocurrido en los pacientes Ambulatorios?**

In [141]:
data[data.defuncion & (data.tipo_paciente == "AMBULATORIO")].defuncion.value_counts()

True    9327
Name: defuncion, dtype: int64